## Walk through of a sample PCL code:

###  Lesson 3 Snapshot copy

[snap.py](../../snap.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
ONTAP 9.13.1 REST API Python Sample Scripts
Author: Vish Hulikal
Purpose: Script to create snapshot using the netapp_ontap library.
         It will also create a policy for the snapshot.
         It will also create a schedule for the policy.

Usage: python3.11 snap.py [-h] -c CLUSTER -v VOLUME_NAME -s SNAPSHOT_NAME -vs VSERVER_NAME
                    -sc SCHEDULE_NAME, -p POLICY_NAME, [-u API_USER] [-p API_PASS]
create_snap.py: The following arguments are required: -c/--cluster, -v/--volume_name
                -s/--snapshot_name -vs/--vserver_name, -sc/--schedule_name, -sp/--policy_name
"""


#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.aggregate`](https://library.netapp.com/ecmdocs/ECMLP2885777/html/resources/aggregate.html) : Aggregate operations through REST API
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.volume`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/volume.html) 


In [ ]:

import argparse
from getpass import getpass
import logging
from typing import Optional

from netapp_ontap import config, HostConnection, NetAppRestError
from netapp_ontap.resources import Volume, Snapshot, SnapshotPolicy, Schedule


#### Function Definitions

These functions

 are defined:
  - `create_snap`
  - `create_snapshot_policy`
  - `create_schedule`  
  

In [ ]:

def create_snap(vserver_name: str, vol_name: str, snapshot_name: str) -> Optional[Snapshot]:
    """Create a new snapshot with default settings for a given volume"""

    volume = Volume.find(name=vol_name, svm={"name":vserver_name})
    if (volume == None):
        print ("Error: Volume %s not found!" % volume.name)
        return

    snapshot = Snapshot(volume.uuid, name=snapshot_name)

    try:
        snapshot.post()
        print("Snapshot %s created successfullys" % snapshot.name)
        return snapshot
    except NetAppRestError as err:
        print("Error: Snapshot was not created: %s" % err)
    return None


In [ ]:

def create_snapshot_policy(policy_name: str, vserver_name: str, schedule_name: str) -> Optional[SnapshotPolicy]:
    """Creates a Snapshot copy policy"""

    data = {
        'name': policy_name,
        'svm': {'name': vserver_name},
        'comment': 'Snapshot Copy Policy',
        'copies': [{'schedule': {'name': schedule_name}, 'count': '5'}]
    }

    snapshot_policy = SnapshotPolicy(**data)

    try:
        snapshot_policy.post()
        print("Snapshot policy %s created successfully" % snapshot_policy.name)
        return snapshot_policy
    except NetAppRestError as err:
        print("Error: Snapshot policy was not created: %s" % err)
    return None


In [ ]:

def create_schedule(schedule_name: str) -> Optional[Schedule]:
    """Creating a schedule that will be used to create a snapshot policy"""

    try:
        schedule = Schedule(
            name=schedule_name, cron={"days": [1], "hours": [3], "minutes": [15]},
        )
        schedule.post()
        print("Schedule %s created successfully" % schedule.name)
        return schedule
    except NetAppRestError as err:
        print("Error: Schedule was not created: %s" % err)
    return None


#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a new snapshot."
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="API server IP:port details"
    )
    parser.add_argument(
        "-v", "--volume_name", required=True, help="Volume to create"
    )
    parser.add_argument(
        "-s", "--snapshot_name", required=True, help="Snapshot to create"
    )
    parser.add_argument(
        "-sp", "--policy_name", required=True, help="Snapshot Policy to create"
    )
    parser.add_argument(
        "-sc", "--schedule_name", required=True, help="Snapshot Policy Schedule to create"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="VServer name to create Snapshot copy"
    )
    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )
    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    snapshot = create_snap(args.vserver_name, args.volume_name, args.snapshot_name)
    schedule = create_schedule(args.schedule_name)
    policy = create_snapshot_policy(args.policy_name, args.vserver_name, args.schedule_name)


### Verify that the snapshot was created:

In [ ]:
!ssh admin@cluster1 snapshot show -vserver nas_svm -volume Vol1

### Verify that the job schedule was created:

In [ ]:
!ssh admin@cluster1 job schedule show -name NewWeek

### Verify that the snapshot policy was created:

In [ ]:
!ssh admin@cluster1 snapshot policy show -vserver nas_svm 

---

[Back to Module 2 Demo - Notebook:](./Module2_demo.ipynb)